In [1]:
import json
import os
import glob
import re
import pandas as pd
from os import makedirs
from os.path import join, exists
from pathlib import Path
from datetime import datetime, date, timedelta
from pathlib import Path
from pycorenlp import StanfordCoreNLP
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/rmh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [3]:
def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 1000,
                       })
    
    try:
        if len(res['sentences']) == 0:
            return -1
        else:
            return res['sentences'][0]['sentimentValue']
    except:
        return False
    else:
        return 2 #neutral sentiment

In [15]:
def get_and_save_election_data(src_path, target_path, start_date, end_date):

    data = []
    dayrange = range((end_date - start_date).days + 1)
    
    for daycount in dayrange:
        dt = start_date + timedelta(days=daycount)
        datestr = dt.strftime('%Y-%m-%d')
        fname = join(src_path, datestr + '.json')
        
        if Path(fname).is_file():

            with open(fname) as f:
                for hd in json.load(f):
                    score = -1
                    if hd != None:
                        score = int(get_sentiment(hd.lower()))
                    
                    #if score == -1:
                    #    score = 2

                    data.append({'date': datestr, 'score': score})
        else:
            data.append({'date': datestr, 'score': -1})

    os.makedirs(target_path, exist_ok=True)
    
    df = pd.DataFrame(data, columns=['date', 'score'])
    
    mean_score = df['score'].mean(skipna=True)
    df['score']=df.score.mask(df.score == -1,mean_score)
    
    
    if len(df) == 0:
        df2 = df.groupby('date')['score'].mean().to_frame().reset_index()
    
    df2.to_csv(join(target_path, 'headlines.csv'), index=False)

In [13]:
def agg_all(source):
    #start_dt = datetime.today() - timedelta(days=30)
    #end_dt = datetime.today() - timedelta(days=1)
    
    start_dt = date(2020,5,26)
    end_dt = date(2020,6,27)
    
    
    print('cand')
    
    get_and_save_election_data(join('data2','candidates', source,'trump'), 
                             join('data3','candidates', source, 'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','candidates', source,'biden'), 
                             join('data3','candidates', source, 'biden'), 
                             start_dt, end_dt)
    
    print('econ')
    
    get_and_save_election_data(join('data2','economy',source,'trump'), 
                             join('data3','economy', source, 'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','economy',source,'biden'), 
                             join('data3','economy',source,'biden'), 
                             start_dt, end_dt)
    
    print('env')
    
    get_and_save_election_data(join('data2','environment',source,'trump'), 
                             join('data3','environment',source,'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','environment',source,'biden'), 
                             join('data3','environment',source,'biden'), 
                             start_dt, end_dt)
    
    print('party')
    
    get_and_save_election_data(join('data2','party',source,'trump'), 
                             join('data3','party',source,'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','party',source,'biden'), 
                             join('data3','party',source,'biden'), 
                             start_dt, end_dt)
    
    print('health')
    
    get_and_save_election_data(join('data2','health',source,'trump'), 
                             join('data3','health',source,'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','health',source,'biden'), 
                             join('data3','health',source,'biden'), 
                             start_dt, end_dt)
    
    print('imm')
    
    get_and_save_election_data(join('data2','immigration',source,'trump'), 
                             join('data3','immigration',source,'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','immigration',source,'biden'), 
                             join('data3','immigration',source,'biden'), 
                             start_dt, end_dt)
    
    print('job')
    
    get_and_save_election_data(join('data2','job',source,'trump'), 
                             join('data3','job',source,'trump'), 
                             start_dt, end_dt)
    
    get_and_save_election_data(join('data2','job',source,'biden'), 
                             join('data3','job',source,'biden'), 
                             start_dt, end_dt)

In [12]:
agg_all('cnn')

cand
econ
env
party
health
imm
job


In [13]:
agg_all('fox-news')

cand
econ
env
party
health
imm
job


In [16]:
agg_all('guardian')

cand
         date  score
0  2020-05-29      2
1  2020-06-08      2
2  2020-06-09      1
3  2020-06-11      2
4  2020-06-12      2
         date  score
0  2020-05-26      1
1  2020-05-27      3
2  2020-05-27      2
3  2020-05-27      2
4  2020-05-28      3
econ
         date  score
0  2020-06-05      1
1  2020-06-12      1
2  2020-06-22      1
3  2020-06-23      2
4  2020-06-23      2
         date  score
0  2020-06-12      1
env
         date  score
0  2020-05-28      1
1  2020-05-31      1
2  2020-06-04      2
3  2020-06-13      1
4  2020-06-21      1
Empty DataFrame
Columns: [date, score]
Index: []


DataError: No numeric types to aggregate

In [5]:
#combine_and_save_tweet_files(join('data2','raw','candidate','2016','trump'), 
#                             join('data2','agg','candidate','2016'), 
#                             date(2016, 10, 8), date(2016, 11, 7), '2016', 'trump')

In [7]:
#combine_and_save_tweet_files(join('data2','raw','candidate','2016','hillary'), 
#                             join('data2','agg','candidate','2016'), 
#                             date(2016, 10, 8), date(2016, 11, 7), '2016', 'hillary')

In [9]:
#combine_and_save_tweet_files(join('data2','raw','candidate','2012','obama'), 
#                             join('data2','agg','candidate','2012','obama'), 
#                             date(2012, 10, 20), date(2012, 11, 4), '2012', 'obama')

In [11]:
#combine_and_save_tweet_files(join('data2','raw','candidate','2012','romney'), 
#                             join('data2','agg','candidate','2012', 'romney'), 
#                             date(2012, 10, 20), date(2012, 11, 4), '2012', 'romney')